# Common

In [ ]:
def pval_to_star(pvalue):
    if pvalue <= 0.0001:
        return "****"
    elif pvalue <= 0.001:
        return "***"
    elif pvalue <= 0.01:
        return "**"
    elif pvalue <= 0.05:
        return "*"
    return "ns"

# Motoneurons

In [ ]:
def get_label_and_color_lists(mid, tot):    
    
    label_list = [2*i + 1 for i in range(mid)]
    label_list.extend([2*i + 2 for i in range(tot-mid)])
    
    color_list = [rgb_left for i in range(mid)]
    color_list.extend([rgb_right for i in range(int(tot-mid))])
        
    return label_list, color_list




def plot_gc_matrix(gc_motorneurons, loc, gc_type='dff', savepath=None, figname_suffix=None, title=False, save=False):
    n_lags = gc_motorneurons.loc[loc].n_lags
    fish = int(gc_motorneurons.loc[loc].Fish)
    trace = int(gc_motorneurons.loc[loc].Trace)
    n_cells = int(gc_motorneurons.loc[loc].n_cells)
    is_multiv = gc_motorneurons.loc[loc].multivariate
    mid = int(gc_motorneurons.loc[loc].mid)
    
    
    if gc_type == 'raw':
        gc = gc_motorneurons.loc[loc].GC_sig_raw
    elif gc_type == 'dt':
        gc = gc_motorneurons.loc[loc].GC_sig_dt
    elif gc_type == 'f_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_f_smooth
    elif gc_type == 'dfdt_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_dfdt_smooth
    elif gc_type == 'disc_f':
        gc = gc_motorneurons.loc[loc].GC_sig_disc_f
    else:
        if gc_type != 'dff':
            print('Unknown \'gc_type\' param: GC on DF/F is returned.')
        gc = gc_motorneurons.loc[loc].GC_sig
        
        
    plt.figure(figsize=(5, 5))
    gc[gc==0] = np.nan
    n_sig = len([i  for j in gc for i in j if i>0])
    plt.imshow(gc, cmap='YlOrRd')
    # plt.colorbar()
    
    ax = plt.gca()
    ax.set_xticks(np.arange(n_cells))
    ax.set_yticks(np.arange(n_cells))
    
    label_list, color_list = get_label_and_color_lists(mid, n_cells)
    
    ax.set_xticklabels(label_list, size=22) #, fontweight='bold')
    ax.set_yticklabels(label_list, size=22) #, fontweight='bold')
    
    for color, tick in zip(color_list, ax.xaxis.get_major_ticks()):
        tick.label1.set_color(color) 
    for color, tick in zip(color_list, ax.yaxis.get_major_ticks()):
        tick.label1.set_color(color) 

    
    if title:
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
    
    plt.xlabel("to neuron", size=25)
    plt.ylabel("from neuron", size=25)
    
    plt.tight_layout()
    
    if save:
        figname = f'devico_fallani/figures/{savepath}/gc_mat_f{fish}t{trace}{figname_suffix}.svg'
        plt.savefig(figname)
        
    plt.show()


def get_coords(mid, tot):
    """ Calcualte the coordinates of points on a circle. """
    left = get_coords_left(mid)
    right = get_coords_right(tot - mid)
    
    return np.concatenate([left.T, right.T])


def get_coords_left(n_points):
    """ Calculate the coordinates of points on the left semicircle. """
    x = np.linspace(-1/int(n_points/2), -1, int(n_points/2))
    
    if n_points%2 == 1:
        x = np.concatenate((x,[-1.1]))
        
    x = np.concatenate((x, np.linspace(-1, -1/int(n_points/2), int(n_points/2))))    
    y = np.linspace(1, -1, n_points)
            
    return np.array([x, y])



def get_coords_right(n_points):
    """ Calculate the coordinates of points on the right semicircle. """
    x = np.linspace(1/int(n_points/2), 1, int(n_points/2))
    
    if n_points%2 == 1:
        x = np.concatenate((x,[1.1]))
        
    x = np.concatenate((x, np.linspace(1, 1/int(n_points/2), int(n_points/2))))
    y = np.linspace(1, -1, n_points)
            
    return np.array([x, y])


def plot_directed_graph(gc_motorneurons, loc, gc_type='dff', hide_digits=False, figname_suffix=None, title=False, save=False, new_fig=False):
    """ Draw the network: points positions on the circle + GC arrows.
    Circle size proportional to the ipsilateral delta (GC_in - GC_out) 
        
    gc_type: 
    default = 'dff' --> GC on dff
    'raw' --> GC on raw fluorescence
    'dt' --> GC on dff derivative
    etc.
    if anything else --> on dff
    
        --> use whole delta instead of ipsi? /!\ if so need to adjust circle size parameter s
        now: s=np.abs(circle_size[i]*1000+(100*is_multiv))
    """
    textsize = 35
    # cmap = plt.cm.YlOrRd
    cmap = plt.cm.Greys
    
    n_lags = gc_motorneurons.loc[loc].n_lags
    fish = int(gc_motorneurons.loc[loc].Fish)
    trace = int(gc_motorneurons.loc[loc].Trace)
    n_cells = int(gc_motorneurons.loc[loc].n_cells)
    mid = int(gc_motorneurons.loc[loc].mid)
    is_multiv = gc_motorneurons.loc[loc].multivariate
    
    if gc_type == 'raw':
        gc = gc_motorneurons.loc[loc].GC_sig_raw
    elif gc_type == 'dt':
        gc = gc_motorneurons.loc[loc].GC_sig_dt
    elif gc_type == 'f_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_f_smooth
    elif gc_type == 'dfdt_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_dfdt_smooth
    elif gc_type == 'disc_f':
        gc = gc_motorneurons.loc[loc].GC_sig_disc_f
    elif gc_type == 'new_thresh':
        gc = gc_motorneurons.loc[loc].GC_sig_new_thresh
    else:
        if gc_type != 'dff':
            print('Unknown \'gc_type\' param: GC on DF/F is returned.')
        gc = gc_motorneurons.loc[loc].GC_sig
    
    gc_ipsi_left = gc[:mid,:mid]
    gc_ipsi_right = gc[mid:, mid:]

    gc_contra_from_left = gc[:mid, mid:]
    gc_contra_from_right = gc[mid:, :mid]

    d_in = np.nansum(gc, axis=0)
    d_out = np.nansum(gc, axis=1)
    
    d_in_ipsi_left = np.nansum(gc_ipsi_left, axis=0)
    d_out_ipsi_left = np.nansum(gc_ipsi_left, axis=1)

    d_in_ipsi_right = np.nansum(gc_ipsi_right, axis=0)
    d_out_ipsi_right = np.nansum(gc_ipsi_right, axis=1)

    d_in_contra_right = np.nansum(gc_contra_from_left, axis=0) # left to right, in right
    d_out_contra_left = np.nansum(gc_contra_from_left, axis=1) # left to right, out left

    d_in_contra_left = np.nansum(gc_contra_from_right, axis=0) # right to left, in left
    d_out_contra_right = np.nansum(gc_contra_from_right, axis=1) # right to left, out right


    # Node centrality (or importance)
    delta = d_in - d_out
    delta_ipsi_left = d_out_ipsi_left - d_in_ipsi_left
    delta_ipsi_right = d_out_ipsi_right - d_in_ipsi_right
    delta_contra_left = d_out_contra_left - d_in_contra_left
    delta_contra_right = d_out_contra_right - d_in_contra_right

    circle_centers = get_coords(mid, n_cells)
    circle_size = np.concatenate([delta_ipsi_left, delta_ipsi_right])
    
    max_circle_size = np.max(np.abs(circle_size))
    circle_size = circle_size / max_circle_size
    
    if new_fig:
        plt.figure(figsize=(6, 6)) 
        
    for i, center in enumerate(circle_centers):
        if circle_size[i] < 0: 
            color = 'navy'
        elif circle_size[i] == 0:
            color = 'purple'
        else:
            color = 'firebrick'
        plt.scatter(center[0], center[1], s=100+np.abs(circle_size[i])*400, c=color)

    plt.axis("equal")

    gc_flat = gc.flatten()
    gc_flat[np.isnan(gc_flat)] = 0
    cells_sorted = np.argsort(gc_flat)
    
    gc_max = gc_flat[cells_sorted[-1]]
    gc_min = gc_flat[cells_sorted[0]]
    
    for cell in cells_sorted:
        gc_cell = gc_flat[cell]
        if gc_cell > 0:
            prop = (gc_cell-gc_min) / (gc_max-gc_min)
            color = cmap(prop) 
            width = prop * 0.05 
            cell_from = int(cell/len(circle_centers))
            cell_to = cell % len(circle_centers) 
            plt.arrow(circle_centers[cell_from,0], circle_centers[cell_from,1], 
                      circle_centers[cell_to,0]-circle_centers[cell_from,0], 
                      circle_centers[cell_to,1]-circle_centers[cell_from,1], 
                      color=color, width=width, length_includes_head=True) # , 
                      # alpha= 0.8)# np.min([np.max([0.5, prop]), 0.8])) # alpha between 0.5 and 0.8
            
            
    label_list, color_list = get_label_and_color_lists(mid, n_cells)
    
    if not hide_digits:
        for label, color, pos in zip(label_list, color_list, circle_centers):
            plt.text(pos[0] + np.sign(pos[0])*0.4, pos[1], label, va='center', ha='center', color=color, size=textsize, fontweight='bold')

    if title:
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}", size=35)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}", size=35)
    plt.axis('off')
    
    #plt.autoscale()
    plt.tight_layout()
    
    if save:
        figname = f'devico_fallani/figures/fig2_n_lags/gc_graph_f{fish}t{trace}{figname_suffix}.svg'
        plt.savefig(figname)
        
        
        
def plot_directed_graph_fantom(gc_motorneurons_clean, gc_motorneurons, loc, idx_to_rmv, gc_type='dff', title=False):

    # cmap = plt.cm.YlOrRd
    cmap = plt.cm.Greys
    
    n_lags = gc_motorneurons.loc[loc].n_lags
    fish = gc_motorneurons.loc[loc].Fish
    trace = gc_motorneurons.loc[loc].Trace

    n_cells = int(gc_motorneurons.loc[loc].n_cells)
    mid = int(gc_motorneurons.loc[loc].mid)
    n_cells_clean = int(gc_motorneurons_clean.loc[loc].n_cells)
    mid_clean = int(gc_motorneurons_clean.loc[loc].mid)
    is_multiv = gc_motorneurons.loc[loc].multivariate
    
    if gc_type == 'raw':
        gc = gc_motorneurons_clean.loc[loc].GC_sig_raw
    elif gc_type == 'dt':
        gc = gc_motorneurons_clean.loc[loc].GC_sig_dt
    elif gc_type == 'f_smooth':
        gc = gc_motorneurons_clean.loc[loc].GC_sig_f_smooth
    elif gc_type == 'dfdt_smooth':
        gc = gc_motorneurons_clean.loc[loc].GC_sig_dfdt_smooth
    elif gc_type == 'disc_f':
        gc = gc_motorneurons_clean.loc[loc].GC_sig_disc_f
    else:
        if gc_type != 'dff':
            print('Unknown \'gc_type\' param: GC on DF/F is returned.')
        gc = gc_motorneurons_clean.loc[loc].GC_sig
    
    gc_ipsi_left = gc[:mid_clean, :mid_clean]
    gc_ipsi_right = gc[mid_clean:, mid_clean:]

    gc_contra_from_left = gc[:mid_clean, mid_clean:]
    gc_contra_from_right = gc[mid_clean:, :mid_clean]

    d_in = np.nansum(gc, axis=0)
    d_out = np.nansum(gc, axis=1)
    
    d_in_ipsi_left = np.nansum(gc_ipsi_left, axis=0)
    d_out_ipsi_left = np.nansum(gc_ipsi_left, axis=1)

    d_in_ipsi_right = np.nansum(gc_ipsi_right, axis=0)
    d_out_ipsi_right = np.nansum(gc_ipsi_right, axis=1)

    d_in_contra_right = np.nansum(gc_contra_from_left, axis=0) # left to right, in right
    d_out_contra_left = np.nansum(gc_contra_from_left, axis=1) # left to right, out left

    d_in_contra_left = np.nansum(gc_contra_from_right, axis=0) # right to left, in left
    d_out_contra_right = np.nansum(gc_contra_from_right, axis=1) # right to left, out right


    # Node centrality (or importance)
    delta = d_in - d_out
    delta_ipsi_left = d_out_ipsi_left - d_in_ipsi_left
    delta_ipsi_right = d_out_ipsi_right - d_in_ipsi_right
    delta_contra_left = d_out_contra_left - d_in_contra_left
    delta_contra_right = d_out_contra_right - d_in_contra_right

    circle_centers = get_coords(mid, n_cells)
    circle_size = np.concatenate([delta_ipsi_left, delta_ipsi_right])
    
    a = 0
    
    for i, center in enumerate(circle_centers):
        if i not in idx_to_rmv:
            if circle_size[a] < 0: 
                color = 'navy'
                size = np.abs(circle_size[a])*1000+(400*is_multiv)
            elif circle_size[a] == 0:
                color = 'purple'
                size = 200
            else:
                color = 'firebrick'
                size = np.abs(circle_size[a])*1000+(400*is_multiv)
            a = a + 1
        else:
            color = 'dimgray'
            size = 200 #+(200*is_multiv)
        plt.scatter(center[0], center[1], s=size, c=color)

    plt.axis("equal")
    
    circle_centers_clean = np.delete(circle_centers, idx_to_rmv, axis=0)
    
    gc_flat = gc.flatten()
    gc_flat[np.isnan(gc_flat)] = 0
    cells_sorted = np.argsort(gc_flat)
    
    gc_max = gc_flat[cells_sorted[-1]]
    gc_min = gc_flat[cells_sorted[0]]
    
    for cell in cells_sorted:
        gc_cell = gc_flat[cell]
        if gc_cell > 0:
            prop = (gc_cell-gc_min) / (gc_max-gc_min)
            color = cmap(prop) 
            width = prop * 0.05 
            cell_from = int(cell/len(circle_centers_clean))
            cell_to = cell % len(circle_centers_clean) 
            plt.arrow(circle_centers_clean[cell_from,0], circle_centers_clean[cell_from,1], 
                      circle_centers_clean[cell_to,0]-circle_centers_clean[cell_from,0], 
                      circle_centers_clean[cell_to,1]-circle_centers_clean[cell_from,1], 
                      color=color, width=width, length_includes_head=True)
            
            
    label_list, color_list = get_label_and_color_lists(mid, n_cells)
    
    for label, color, pos in zip(label_list, color_list, circle_centers):
        plt.text(pos[0] + np.sign(pos[0])*0.4, pos[1], label, va='center', ha='center', color=color, size=textsize, fontweight='bold')
        
    if title:
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}", size=35)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}", size=35)
            
    plt.axis('off')
    plt.tight_layout()

    
    
    

In [ ]:
def plot_ideal_gc_matrix(n_cells, mid, is_multiv=False, savepath=None, figname_suffix=None, title=False, save=False):
    gc = np.zeros((n_cells, n_cells))
    if is_multiv:
        for i in range(n_cells-1):
            if i != mid-1:
                gc[i, i+1] = 1
    else:           
        for i in range(n_cells):
            if i < mid:
                for j in range(i+1, mid):
                    gc[i, j] = 1
            else:
                for j in range(i+1, n_cells):
                    gc[i, j] = 1
        
    plt.figure(figsize=(5, 5))
    gc[gc==0] = np.nan
    n_sig = len([i  for j in gc for i in j if i>0])
    plt.imshow(gc, cmap='RdYlBu')
    
    ax = plt.gca()
    ax.set_xticks(np.arange(n_cells))
    ax.set_yticks(np.arange(n_cells))
    
    label_list, color_list = get_label_and_color_lists(mid, n_cells)
    
    ax.set_xticklabels(label_list, size=22) #, fontweight='bold')
    ax.set_yticklabels(label_list, size=22) #, fontweight='bold')
    
    for color, tick in zip(color_list, ax.xaxis.get_major_ticks()):
        tick.label1.set_color(color) 
    for color, tick in zip(color_list, ax.yaxis.get_major_ticks()):
        tick.label1.set_color(color) 

    
    if title:
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
    
    plt.xlabel("to neuron", size=25)
    plt.ylabel("from neuron", size=25)
    
    plt.tight_layout()
    
    if save:
        figname = f'devico_fallani/figures/{savepath}/gc_mat_f{fish}t{trace}{figname_suffix}.svg'
        plt.savefig(figname)
        
    plt.show()
    
    
def plot_ideal_graph(n_cells, mid, is_multiv=False, figname_suffix=None, title=False, save=False):
    """ Draw the ideal network: points positions on the circle + GC arrows.
    """
    textsize = 35
    cmap = plt.cm.YlOrRd
    
    gc = np.zeros((n_cells, n_cells))
    if is_multiv:
        for i in range(n_cells-1):
            if i != mid-1:
                gc[i, i+1] = 1
    else:           
        for i in range(n_cells):
            if i < mid:
                for j in range(i+1, mid):
                    gc[i, j] = 1
            else:
                for j in range(i+1, n_cells):
                    gc[i, j] = 1


    gc_ipsi_left = gc[:mid,:mid]
    gc_ipsi_right = gc[mid:, mid:]

    gc_contra_from_left = gc[:mid, mid:]
    gc_contra_from_right = gc[mid:, :mid]

    d_in = np.nansum(gc, axis=0)
    d_out = np.nansum(gc, axis=1)
    
    d_in_ipsi_left = np.nansum(gc_ipsi_left, axis=0)
    d_out_ipsi_left = np.nansum(gc_ipsi_left, axis=1)

    d_in_ipsi_right = np.nansum(gc_ipsi_right, axis=0)
    d_out_ipsi_right = np.nansum(gc_ipsi_right, axis=1)

    d_in_contra_right = np.nansum(gc_contra_from_left, axis=0) # left to right, in right
    d_out_contra_left = np.nansum(gc_contra_from_left, axis=1) # left to right, out left

    d_in_contra_left = np.nansum(gc_contra_from_right, axis=0) # right to left, in left
    d_out_contra_right = np.nansum(gc_contra_from_right, axis=1) # right to left, out right


    # Node centrality (or importance)
    delta = d_in - d_out
    delta_ipsi_left = d_out_ipsi_left - d_in_ipsi_left
    delta_ipsi_right = d_out_ipsi_right - d_in_ipsi_right
    delta_contra_left = d_out_contra_left - d_in_contra_left
    delta_contra_right = d_out_contra_right - d_in_contra_right

    circle_centers = get_coords(mid, n_cells)
    circle_size = np.concatenate([delta_ipsi_left, delta_ipsi_right])
    
    max_circle_size = np.max(np.abs(circle_size))
    circle_size = circle_size / max_circle_size
  
    for i, center in enumerate(circle_centers):
        if circle_size[i] < 0: 
            color = 'navy'
        elif circle_size[i] == 0:
            color = 'purple'
        else:
            color = 'firebrick'
        plt.scatter(center[0], center[1], s=100+np.abs(circle_size[i])*300, c=color)

    plt.axis("equal")

    gc_flat = gc.flatten()
    gc_flat[np.isnan(gc_flat)] = 0
    cells_sorted = np.argsort(gc_flat)
    
    gc_max = gc_flat[cells_sorted[-1]]
    gc_min = gc_flat[cells_sorted[0]]
    
    for cell in cells_sorted:
        gc_cell = gc_flat[cell]
        if gc_cell > 0:
            prop = (gc_cell-gc_min) / (gc_max-gc_min)
            color = cmap(prop) 
            width = 0.025
            cell_from = int(cell/len(circle_centers))
            cell_to = cell % len(circle_centers) 
            plt.arrow(circle_centers[cell_from,0], circle_centers[cell_from,1], 
                      circle_centers[cell_to,0]-circle_centers[cell_from,0], 
                      circle_centers[cell_to,1]-circle_centers[cell_from,1], 
                      color='dimgray', width=width, length_includes_head=True, alpha=0.8)
            
            
    label_list, color_list = get_label_and_color_lists(mid, n_cells)
    
    for label, color, pos in zip(label_list, color_list, circle_centers):
        plt.text(pos[0] + np.sign(pos[0])*0.4, pos[1], label, va='center', ha='center', color=color, size=textsize, fontweight='bold')
        
    if title:
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}", size=35)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}", size=35)
    plt.axis('off')
    
    plt.autoscale()
    plt.tight_layout()
    
    if save:
        figname = f'devico_fallani/figures/fig2_n_lags/gc_graph_f{fish}t{trace}{figname_suffix}.svg'
        plt.savefig(figname)

# Hindbrain

In [ ]:
# PLOT CELLS ON BRAIN BACKGROUND
def plot_background_and_cells(cell_centers, subset_neurons, background, subplots=False, invert=False):
    """ Plot the brain image of the plane and corresponding cells (of a fish-trial pair).
    To make two subplots (for info flow plots) use subplots=True.
    To only draw background without cells, use scatter=False (for emit/receive plots).
    For fish 2, 5, 6, 8: use invert=True
    cmap=plt.cm.gist_yarg: to have a background more white than black.
    vmax=np.max(background)/2: to increase contrast in background.
    """

    if subplots:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
        ax1.imshow(background, cmap=plt.cm.gist_yarg)
        ax1.set_title('Rostral to caudal', fontsize=25)
        ax2.imshow(background, cmap=plt.cm.gist_yarg)
        ax2.set_title('Caudal to rostral', fontsize=25)
        if invert:
            ax1.scatter(512 - cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            ax2.scatter(512 - cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            ax1.scatter(512 - cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
            ax2.scatter(512 - cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
        else:
            ax1.scatter(cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            ax2.scatter(cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            ax1.scatter(cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
            ax2.scatter(cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
        
        res = fig, (ax1, ax2)
    else:
        fig = plt.figure(figsize=(10,6))
        plt.imshow(background, cmap=plt.cm.gist_yarg, aspect='equal', vmax=np.max(background)/2)
        if invert:
            plt.scatter(512 - cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            plt.scatter(512 - cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
        else:
            plt.scatter(cell_centers[:,0], cell_centers[:,1], color='silver', edgecolor='black', s=100)
            plt.scatter(cell_centers[subset_neurons,0], cell_centers[subset_neurons,1], color='crimson', edgecolor='black', s=100)
        res = fig
    plt.axis('off')
    plt.tight_layout()
    
    return res

def plot_gc_matrix_HB(gc_motorneurons, loc, gc_type='dff_8', savepath=None, figname_suffix=None, title=False, save=False):
    n_lags = gc_motorneurons.loc[loc].n_lags
    n_cells = int(gc_motorneurons.loc[loc].n_cells)
    is_multiv = gc_motorneurons.loc[loc].multivariate
    
    
    if gc_type == 'raw':
        gc = gc_motorneurons.loc[loc].GC_sig_raw
    elif gc_type == 'dt':
        gc = gc_motorneurons.loc[loc].GC_sig_dt
    elif gc_type == 'f_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_f_smooth
    elif gc_type == 'dfdt_smooth':
        gc = gc_motorneurons.loc[loc].GC_sig_dfdt_smooth
    elif gc_type == 'disc_f':
        gc = gc_motorneurons.loc[loc].GC_sig_disc_f
    else:
        if gc_type != 'dff_8':
            print('Unknown \'gc_type\' param: GC on DF/F is returned.')
        gc = gc_motorneurons.loc[loc].GC_sig
        
        
    plt.figure(figsize=(5, 5))
    gc[gc==0] = np.nan
    n_sig = len([i  for j in gc for i in j if i>0])
    plt.imshow(gc, cmap='YlOrRd')
    
    if title: 
        fish = int(gc_motorneurons.loc[loc].Fish)
        trace = int(gc_motorneurons.loc[loc].Trace)
        if is_multiv:
            plt.suptitle(f"Fish {fish}, trace {trace} - Multivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
        else:
            plt.suptitle(f"Fish {fish}, trace {trace} - Bivariate GC n_lags={n_lags}, {n_sig} significant links", size=20)
    
    plt.xlabel("to neuron", size=25)
    plt.ylabel("from neuron", size=25)
    
    plt.tight_layout()
    
    if save:
        figname = f'figures/{savepath}/gc_mat_f{fish}t{trace}{figname_suffix}.svg'
        plt.savefig(figname)
        
    plt.show()

def plot_drive(cell_centers, subset_neurons, drive, background, default_color='silver', cst=1, invert=False):
    """ Plot the brain image of the plane and corresponding cells (of a fish-trial pair).
    Cell size proportional to its drive.
    """

    fig = plt.figure(figsize=(10,6))
    plt.imshow(background, cmap=plt.cm.gist_yarg, aspect='equal', vmax=np.max(background)/2)
    next_sub_idx = 0
        
    if invert:
        for j in range(len(cell_centers)):
            if next_sub_idx < len(subset_neurons) and j == subset_neurons[next_sub_idx]:
                color = 'crimson'
                next_sub_idx = next_sub_idx+1
            else:
                color = default_color
            plt.scatter(512 - cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=100*(int(drive[j]*cst)+1), alpha=0.8)
    else:
        for j in range(len(cell_centers)):
            if next_sub_idx < len(subset_neurons) and j == subset_neurons[next_sub_idx]:
                color = 'crimson'
                next_sub_idx = next_sub_idx+1
            else:
                color = default_color
            plt.scatter(cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=100*(int(drive[j]*cst)+1), alpha=0.8)
    plt.axis('off')
    
    return fig



def plot_background_and_cells_drive(cell_centers, background, drive, cst, color='crimson', subplots=False, invert=False):
    """ Plot the brain image of the plane and corresponding cells (of a fish-trial pair).
    To make two subplots (for info flow plots) use subplots=True.
    To only draw background without cells, use scatter=False (for emit/receive plots).
    For fish 2, 5, 6, 8: use invert=True
    cmap=plt.cm.gist_yarg: to have a background more white than black.
    vmax=np.max(background)/2: to increase contrast in background.
    Size proportional to drive.
    """
    ori_color = color
    
    if subplots:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
        ax1.imshow(background, cmap=plt.cm.gist_yarg)
        ax1.set_title('Rostral to caudal', fontsize=18)
        ax2.imshow(background, cmap=plt.cm.gist_yarg)
        ax2.set_title('Caudal to rostral', fontsize=18)
        if invert:
            for j in range(len(cell_centers)):
                s = 10*(int(drive[j]*cst) + np.sign(drive[j]))
                if s < 0:
                    s = np.abs(s)
                    color = 'navy'
                elif s == 0:
                    s = 100
                    color='purple'
                ax1.scatter(512 - cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=s)
                ax2.scatter(512 - cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=s)
                color = ori_color
        else:
            for j in range(len(cell_centers)):
                s = 100*(int(drive[j]*cst) + np.sign(drive[j]))
                if s < 0:
                    s = np.abs(s)
                    color = 'navy'
                elif s == 0:
                    s = 100
                    color='purple'
                ax1.scatter(cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=s)
                ax2.scatter(cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=s)
                color = ori_color
        
        res = fig, (ax1, ax2)
    else:
        fig = plt.figure(figsize=(10,6))
        plt.imshow(background, cmap=plt.cm.gist_yarg, aspect='equal', vmax=np.max(background)/2)
        if invert:
            for j in range(len(cell_centers)):
                plt.scatter(512 - cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=10*(int(drive[j]*cst)+1))
        else:
            for j in range(len(cell_centers)):
                plt.scatter(cell_centers[j,0], cell_centers[j,1], color=color, edgecolor='black', s=10*(int(drive[j]*cst)+1))
        res = fig
    plt.axis('off')
    
    return res



def plot_flow_medial(gc_medial, subset_cell_centers, background, fish, zoom=False, drive_type='sum_out', xlims=[200,360], ylims=[85,165], cst=100, title='Subset'):
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
        
    if drive_type == 'sum_out':
        drive = np.nansum(gc_medial, axis=1)
    elif drive_type == 'difference':
        drive = np.nansum(gc_medial, axis=1) - np.nansum(gc_medial, axis=0)
    elif drive_type == 'sum_in':
        drive = np.nansum(gc_medial, axis=0)
    else:
        print('unknown drive_type. using sum out.')
        drive = np.nansum(gc_medial, axis=1)
    
    fig, (ax1, ax2) = plot_background_and_cells_drive(subset_cell_centers, background, drive, cst, subplots=True, invert=invert)

    
    for i_from in range(len(subset_cell_centers)):
        for i_to in range(len(subset_cell_centers)):
            if gc_medial[i_from, i_to] > 0:
                
                x_from = subset_cell_centers[i_from, 0]
                y_from = subset_cell_centers[i_from, 1]

                x_to = subset_cell_centers[i_to, 0]
                y_to = subset_cell_centers[i_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to
                
                
                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', width=0.5, alpha=0.5)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', width=0.5, alpha=0.5) 
                

    ax1.axis('off')
    ax2.axis('off')
    if zoom:
        ax1.set_xlim([200,360])
        ax1.set_ylim([85,165])
        ax2.set_xlim([200,360])
        ax2.set_ylim([85,165])
    # plt.suptitle(title + ' neurons only', size=20)
    return fig, (ax1, ax2)



def plot_info_flow_subset(gc_subset, cell_centers, subset_neurons, background, fish, title='Subset'):
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
    fig, (ax1, ax2) = plot_background_and_cells(cell_centers, subset_neurons, background, subplots=True, invert=invert)

    for i_from, roi_from in enumerate(subset_neurons):
        for i_to, roi_to in enumerate(subset_neurons):
            if gc_subset[i_from, i_to] > 0:
                
                x_from = cell_centers[roi_from, 0]
                y_from = cell_centers[roi_from, 1]

                x_to = cell_centers[roi_to, 0]
                y_to = cell_centers[roi_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to
                
                # x increasing from left to right of plot
                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5) 
                

    ax1.axis('off')
    ax2.axis('off')
    # plt.suptitle(title + ' neurons only', size=20)
    return fig, (ax1, ax2)


def plot_info_flow_swim_only(gc, cell_centers, swim_neurons, background, fish):
    """ Plot the top k strongest rostral to caudal and caudal to rostral GC links.
    """
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
    fig, (ax1, ax2) = plot_background_and_cells(cell_centers, swim_neurons, background, subplots=True, invert=invert)

    for roi_from in swim_neurons:
        for roi_to in swim_neurons:
            if gc[roi_from, roi_to] > 0:
            
                x_from = cell_centers[roi_from, 0]
                y_from = cell_centers[roi_from, 1]

                x_to = cell_centers[roi_to, 0]
                y_to = cell_centers[roi_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to

                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5) 
                

    ax1.axis('off')
    ax2.axis('off')
    plt.suptitle('Swim neurons only', size=20)
    return fig, (ax1, ax2)


def plot_info_flow_swim_involved(gc, cell_centers, swim_neurons, background, fish):
    """ Plot the top k strongest rostral to caudal and caudal to rostral GC links.
    """
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
    fig, (ax1, ax2) = plot_background_and_cells(cell_centers, swim_neurons, background, subplots=True, invert=invert)

    for roi_from in swim_neurons:
        for roi_to in range(len(cell_centers)):
            if gc[roi_from, roi_to] > 0:
            
                x_from = cell_centers[roi_from, 0]
                y_from = cell_centers[roi_from, 1]

                x_to = cell_centers[roi_to, 0]
                y_to = cell_centers[roi_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to

                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', alpha=0.5) 
                
    for roi_from in range(len(cell_centers)):
        for roi_to in swim_neurons:
            
            if gc[roi_from, roi_to] > 0:
            
                x_from = cell_centers[roi_from, 0]
                y_from = cell_centers[roi_from, 1]
                x_to = cell_centers[roi_to, 0]
                y_to = cell_centers[roi_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to

                if x_from >= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='navy', alpha=0.5)
                if x_from <= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='navy', alpha=0.5)


    ax1.axis('off')
    ax2.axis('off')
    plt.suptitle('Swim neurons involved', size=20)

    return fig, (ax1, ax2)


def plot_info_flow_topk(GC, cell_centers, swim_neurons, background, fish, k=100):
    """ Plot the top k strongest rostral to caudal and caudal to rostral GC links.
    """
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
    fig, (ax1, ax2) = plot_background_and_cells(cell_centers, swim_neurons, background, subplots=True, invert=invert)
    GC_flat = GC.flatten()
    GC_flat[np.isnan(GC_flat)] = 0
    GC_sorted = np.argsort(GC_flat)
    topK = GC_sorted[-k:]

    for roi in topK:
        if GC_flat[roi] > 0:
            roi_from = roi % len(cell_centers) # from
            roi_to = int(roi/len(cell_centers)) # to
            x_from = cell_centers[roi_from, 0]
            y_from = cell_centers[roi_from, 1]
            x_to = cell_centers[roi_to, 0]
            y_to = cell_centers[roi_to, 1]

            if invert:
                x_from = 512 - x_from
                x_to = 512 - x_to

            if x_from <= x_to: # rostral to caudal
                ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from)
            if x_from >= x_to: # caudal to rostral
                ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from) 

            
    ax1.axis('off')
    ax2.axis('off')    
    plt.suptitle(f'top {k} links', size=20)

    return fig, (ax1, ax2)


# medial neurons to all neurons map
def plot_info_flow_SR_to_all(gc_SR_to_all, fish, cell_centers, background, swim_neurons, medial_neurons):
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
        
        
    swim_mask = np.zeros(len(cell_centers), dtype=bool)
    swim_mask[swim_neurons] = True
    swim_centers = cell_centers[swim_mask]

    notswim_mask = np.ones(len(cell_centers), dtype=bool)
    notswim_mask[swim_neurons] = False
    notswim_centers = cell_centers[notswim_mask]



    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
    ax1.imshow(background, cmap=plt.cm.gist_yarg)
    ax1.set_title('Rostral to caudal', fontsize=25)
    ax2.imshow(background, cmap=plt.cm.gist_yarg)
    ax2.set_title('Caudal to rostral', fontsize=25)

    ax1.scatter(512 - notswim_centers[:,0], notswim_centers[:,1], color='silver', edgecolor='black', s=200)
    ax2.scatter(512 - notswim_centers[:,0], notswim_centers[:,1], color='silver', edgecolor='black', s=200)

    for n in swim_centers:
        n[0] = 512 - n[0]
        if n[0] > 200 and n[0] < 360 and n[1] > 85 and n[1] < 165:
            ax1.scatter(n[0], n[1], color='crimson', edgecolor='black', s=200)
            ax2.scatter(n[0], n[1], color='crimson', edgecolor='black', s=200)
        else:
            ax1.scatter(n[0], n[1], color='navy', edgecolor='black', s=200)
            ax2.scatter(n[0], n[1], color='navy', edgecolor='black', s=200)

    gc_max = np.nanmax(gc_SR_to_all)
    gc_min = np.nanmin(gc_SR_to_all)
    invert=True

    for i_from in range(len(medial_neurons)):
        for i_to in range(len(cell_centers)):
            if gc_SR_to_all[i_from, i_to] > 0:

                x_from = cell_centers[medial_neurons[i_from], 0]
                y_from = cell_centers[medial_neurons[i_from], 1]

                x_to = cell_centers[i_to, 0]
                y_to = cell_centers[i_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to



                width = 2*(gc_SR_to_all[i_from, i_to] - gc_min) / (gc_max - gc_min)


                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', length_includes_head=True, width=width, alpha=0.6)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', length_includes_head=True, width=width, alpha=0.6) 


    ax1.axis('off')
    ax2.axis('off')

    plt.tight_layout()
    
    
    
def plot_flow_medial_strength(gc_medial, subset_cell_centers, background, fish, zoom=False, drive_type='sum_out', xlims=[200,360], ylims=[85,165], cst=100, title='Subset'):
    """ Arrow size proportional to GC value (strength) """
    if fish in [2, 5, 6, 8]:
        invert = True
    else:
        invert = False
        
    if drive_type == 'sum_out':
        drive = np.nansum(gc_medial, axis=1)
    elif drive_type == 'difference':
        drive = np.nansum(gc_medial, axis=1) - np.nansum(gc_medial, axis=0)
    elif drive_type == 'sum_in':
        drive = np.nansum(gc_medial, axis=0)
    else:
        print('unknown drive_type. using sum out.')
        drive = np.nansum(gc_medial, axis=1)
    
    fig, (ax1, ax2) = plot_background_and_cells_drive(subset_cell_centers, background, drive, cst, subplots=True, invert=invert)
    
    gc_max = np.nanmax(gc_medial)
    gc_min = np.nanmin(gc_medial)
    
    for i_from in range(len(subset_cell_centers)):
        for i_to in range(len(subset_cell_centers)):
            if gc_medial[i_from, i_to] > 0:
                
                x_from = subset_cell_centers[i_from, 0]
                y_from = subset_cell_centers[i_from, 1]

                x_to = subset_cell_centers[i_to, 0]
                y_to = subset_cell_centers[i_to, 1]

                if invert:
                    x_from = 512 - x_from
                    x_to = 512 - x_to
                
                
                
                width = 2*(gc_medial[i_from, i_to] - gc_min) / (gc_max - gc_min)
                
                
                if x_from <= x_to: # rostral to caudal
                    ax1.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', length_includes_head=True, width=width, alpha=0.6)
                if x_from >= x_to: # caudal to rostral
                    ax2.arrow(x_from, y_from, x_to-x_from, y_to-y_from, color='crimson', length_includes_head=True, width=width, alpha=0.6) 
                

    ax1.axis('off')
    ax2.axis('off')
    if zoom:
        ax1.set_xlim([200,360])
        ax1.set_ylim([85,165])
        ax2.set_xlim([200,360])
        ax2.set_ylim([85,165])
    # plt.suptitle(title + ' neurons only', size=20)
    return fig, (ax1, ax2)